In [1]:
import polars as pl 
import numpy as np
from eda_utils import _f_score
from sklearn.datasets import make_classification
from sklearn.feature_selection import f_regression
import pandas as pd
from time import perf_counter
import os 

In [2]:
# create some data
orig_x, orig_y = make_classification(n_samples = 1_000_000, n_features = 50, n_informative = 20, n_redundant = 30)

In [3]:
df_pl = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))
df_pd = df_pl.to_pandas()

In [4]:
def mrmr(df:pl.DataFrame, target:str, k:int) -> list[str]:

    features = df.columns
    features.remove(target)

    start = perf_counter()
    f_scores = _f_score(df, target, features)
    end_1 = perf_counter()
    print(f"Spent {end_1 - start:.2f}s to compute f score.")

    df_scaled = df.select(features).with_columns(
        (pl.col(f) - pl.col(f).mean())/pl.col(f).std() for f in features
    )

    cumulating_sum = np.zeros(len(features)) # For each feature at index i, we keep a cumulating sum
    top_idx = np.argmax(f_scores)
    selected_features = [features[top_idx]]
    for j in range(1, k): 
        argmax = -1
        current_max = -1
        last_selected = selected_features[-1]
        for i,f in enumerate(features):
            if f not in selected_features:
                # Left = cumulating sum of abs corr
                # Right = abs correlation btw last_selected and f
                cumulating_sum[i] += np.abs((df_scaled.get_column(last_selected)*df_scaled.get_column(f)).mean())
                denominator = cumulating_sum[i] / j
                new_score = f_scores[i] / denominator
                if new_score > current_max:
                    current_max = new_score
                    argmax = i

        selected_features.append(features[argmax])

    end_2 = perf_counter()
    print(f"Spent {end_2 - end_1:.2f}s to compute mrmr.")
    return selected_features
    

In [5]:
def mrmr_medium(df:pd.DataFrame, target:str, k:int) -> list[str]:

    features = list(df.columns)
    features.remove(target)

    X = df[features]
    y = df[target]

    # compute F-statistics and initialize correlation matrix
    start = perf_counter()
    F = pd.Series(f_regression(X, y)[0], index = X.columns)
    end_1 = perf_counter()
    print(f"Spent {end_1 - start:.2f}s to compute f score.")

    corr = pd.DataFrame(.00001, index = X.columns, columns = X.columns)

    # initialize list of selected features and list of excluded features
    selected = []
    not_selected = X.columns.to_list()

    # repeat K times
    for i in range(k):
    
        # compute (absolute) correlations between the last selected feature and all the (currently) excluded features
        if i > 0:
            last_selected = selected[-1]
            corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
            
        # compute FCQ score for all the (currently) excluded features (this is Formula 2)
        score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
        
        # find best feature, add it to selected and remove it from not_selected
        best = score.index[score.argmax()]
        selected.append(best)
        not_selected.remove(best)

    # print(selected) 
    end_2 = perf_counter()
    print(f"Spent {end_2 - end_1:.2f}s to compute mrmr.")
    return selected 

In [6]:
mrmr(df_pl, "target", 20)

Spent 0.07s to compute f score.
Spent 1.05s to compute mrmr.


['column_5',
 'column_20',
 'column_23',
 'column_3',
 'column_44',
 'column_16',
 'column_48',
 'column_8',
 'column_6',
 'column_27',
 'column_12',
 'column_2',
 'column_47',
 'column_24',
 'column_29',
 'column_4',
 'column_49',
 'column_37',
 'column_33',
 'column_14']

In [7]:
mrmr_medium(df_pd, "target", 20)

Spent 0.15s to compute f score.
Spent 9.91s to compute mrmr.


['column_5',
 'column_20',
 'column_23',
 'column_3',
 'column_44',
 'column_16',
 'column_48',
 'column_8',
 'column_6',
 'column_27',
 'column_12',
 'column_2',
 'column_47',
 'column_24',
 'column_29',
 'column_4',
 'column_49',
 'column_37',
 'column_33',
 'column_14']

In [8]:
from mrmr import mrmr_classif

In [9]:
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    
    features = list(df.columns)
    features.remove(target)

    X = df[features]
    y = df[target]

    # use mrmr classification
    return mrmr_classif(X, y, K = k)

In [10]:
mrmr_package(df_pd, "target", 20)

100%|██████████| 20/20 [00:12<00:00,  1.67it/s]


['column_5',
 'column_20',
 'column_23',
 'column_3',
 'column_44',
 'column_16',
 'column_48',
 'column_8',
 'column_6',
 'column_27',
 'column_12',
 'column_2',
 'column_47',
 'column_24',
 'column_29',
 'column_4',
 'column_49',
 'column_37',
 'column_33',
 'column_14']